In [34]:
from __future__ import print_function, absolute_import
import os
import elastix
import matplotlib.pyplot as plt
import numpy as np
import imageio
import SimpleITK as sitk
import voxelmorph as vxm
import fileinput
import pandas as pd
import cv2

In [2]:
ELASTIX_PATH = os.path.join(r"C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe")
TRANSFORMIX_PATH = os.path.join(r"C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\transformix.exe")
if not os.path.exists(ELASTIX_PATH):
    raise IOError('Elastix cannot be found, please set the correct ELASTIX_PATH.')
if os.path.exists('results') is False:
    os.mkdir('results')
if os.path.exists('results1') is False:
    os.mkdir('results1')
if os.path.exists('atlas_images') is False:
    os.mkdir('atlas_images')

el = elastix.ElastixInterface(elastix_path=ELASTIX_PATH)

In [4]:
parameter_file_path = r"C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt"
parameter_file_path_bspline = r"C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\B-spline1.txt"
data_path = r"C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData" #path to training data folder
atlas_path = r"C:\Users\20192059\Documents\Master\CS in MIA\atlas_images"#path to folder where atlas images are saved
MRI = "mr_bffe.mhd"
atlas  = "prostaat.mhd"

In [5]:
# function for NMI between images
def NMI(img1,img2):
    img1_ = sitk.GetArrayFromImage(img1)
    img2_ = sitk.GetArrayFromImage(img2)
    img2_ = np.reshape(img2_, -1)
    img1_ = np.reshape(img1_, -1)
    normalized_mutual_infor = normalized_mutual_info_score(img1_, img2_)
    nmi = normalized_mutual_infor 
    print(nmi)

In [25]:
nmi_df = pd.DataFrame(columns =['Patient','NMI score'])
patient_list = ["p102","p107","p108","p109","p115","p116","p117","p119","p120","p125","p127","p128","p129","p133","p135"]

In [56]:
# for loop over the patients, in which the selected patient is the gold standard
for i in patient_list:
    #remove existing files in folder
    dir = os.path.join(atlas_path, i) 
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))
    
    #specify validation image
    fixed_image_path = os.path.join(data_path, i, MRI) 
    fixed_image_path_new = os.path.join(data_path, i, atlas) 
    fixed_image_new = imageio.v2.imread(fixed_image_path_new)[40,:,:]

    
    #make list of all patients except validation patient
    training_patients_list = []
    for j in patient_list:
        if i != j:
            training_patients_list.append(j)
    print(training_patients_list)
    
    #loop over training patients 
    for k in training_patients_list:
        moving_image_path = os.path.join(data_path, k, MRI) #training image
        moving_image_path_new = os.path.join(data_path, k, atlas) #training image
        result_path = os.path.join(atlas_path, i)
        
        #affine registration
        el.register(
            fixed_image=fixed_image_path,
            moving_image=moving_image_path,
            parameters=[parameter_file_path],
            output_dir=result_path)#'results')
        
        # alter parameter file after registration
        filename = os.path.join(result_path, 'TransformParameters.0.txt')
        with fileinput.FileInput(filename, backup = '', inplace = True) as f:
            for line in f:
                if("(FinalBSplineInterpolationOrder 3)\n" == line):
                    print("(FinalBSplineInterpolationOrder 0)",end ='\n')
                else:
                    print(line, end ='')
        
        # read images
        result_path_affine = os.path.join(result_path, 'result.0.mhd')
        transformed_moving_image = imageio.v2.imread(result_path_affine)[40,:,:]
        
        #transform atlas image
        tr =elastix.TransformixInterface(parameters=os.path.join(result_path, 'TransformParameters.0.txt'),
        transformix_path=TRANSFORMIX_PATH)
        
        transformed_image_path = tr.transform_image(moving_image_path_new, output_dir=result_path)
        transformed_moving_image_new= imageio.imread(transformed_image_path.replace('dcm', 'tiff'))[40,:,:]
        
        outputFile = os.path.join(atlas_path, i)
        imageio.imwrite(transformed_moving_image_new, outputFile)
        
        #img = cv2.imread(transformed_image_path)
        #print(type(img))
        #os.chdir(result_path)
        #filename = os.path.join("atlas", k+'.tiff')
        #cv2.imwrite(filename, img[40,:,:])
        
        # NMI, select the best images
        img1 = sitk.ReadImage(fixed_image_path,sitk.sitkFloat32)
        img2 = sitk.ReadImage(moving_image_path,sitk.sitkFloat32)
        score = NMI(img1, img2)
        nmi_df = nmi_df.append({'Patient':k, 'NMI score': score}, ignore_index = True)
        
    nmi_df['NMI score'].nlargest(n=5) # define how many atlas images you want to use
    for column in nmi_df:
        top_patients_list = nmi_df['NMI score'].values  
        
    #for l in top_patients_list:
        #result_path_affine = os.path.join(affine_path, i, MRI)

['p107', 'p108', 'p109', 'p115', 'p116', 'p117', 'p119', 'p120', 'p125', 'p127', 'p128', 'p129', 'p133', 'p135']
C:\Users\20192059\Documents\Master\CS in MIA\elastix-5.0.0-win64\elastix.exe -f C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p102\mr_bffe.mhd -m C:\Users\20192059\Documents\Master\CS in MIA\TrainingData\TrainingData\p107\mr_bffe.mhd -p C:\Users\20192059\Documents\Master\CS in MIA\CapitaSelectaTUe-main\Affine.txt -out C:\Users\20192059\Documents\Master\CS in MIA\atlas_images\p102


C:\Users\20192059\.conda\envs\CS\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.


ValueError: Image is not numeric, but str.

In [ ]:
#Bspine registration
        el.register(
            fixed_image=fixed_image_path,
            moving_image=result_path_affine,
            parameters=[parameter_file_path_bspline],
            output_dir='results1')
        
        result_path = os.path.join('results1', 'result.0.mhd')
        
        #fixed_image = imageio.imread(fixed_image_path)[40,:,:]
        #moving_image = imageio.imread(result_path_affine)[40,:,:]
        transformed_moving_image = imageio.imread(result_path)[40,:,:]
        
        filename =  r"C:\Users\20192059\Documents\Master\CS in MIA\results1\TransformParameters.0.txt"
        with fileinput.FileInput(filename, inplace = True) as f:
            for line in f:
                if("(FinalBSplineInterpolationOrder 3)\n" == line):
                    print("(FinalBSplineInterpolationOrder 0)",end ='\n')
                else:
                    print(line, end ='')
                    
        #transform atlas image

        tr_2 =elastix.TransformixInterface(parameters='results1/TransformParameters.0.txt',
        transformix_path=TRANSFORMIX_PATH)
        transformed_image_path_2 = tr_2.transform_image(transformed_image_path, output_dir=r'results1')

        transformed_moving_image_new= imageio.imread(transformed_image_path_2.replace('dcm', 'tiff'))[40,:,:]
        
        #save transformed atlas image
        outputFile = os.path.join(atlas_path, i)
        imageio.write(transformed_moving_image_new, 'tiff', outputFile) #atlas images are saved in the folder with the nme of the validation image
